In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from Custom_CNN import cnn

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_set = torchvision.datasets.CIFAR10(root='../cifar10', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)

test_set = torchvision.datasets.CIFAR10(root='../cifar10', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cpu is available


In [4]:
model = cnn().to(device)
print(model)

cnn(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [6]:
loss_ = []
batch_num = len(train_loader)

for epoch in range(10):
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        input, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    loss_.append(running_loss / batch_num)
    print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(train_loader)))

[1] loss: 1.907
[2] loss: 1.402
[3] loss: 1.239
[4] loss: 1.136
[5] loss: 1.059
[6] loss: 0.999
[7] loss: 0.952
[8] loss: 0.904
[9] loss: 0.867
[10] loss: 0.829


In [7]:
Model_PATH = './Custom_CNN.pt'
torch.save(model.state_dict(), Model_PATH)

In [8]:
model = cnn().to(device)
model.load_state_dict(torch.load(Model_PATH))

<All keys matched successfully>

In [9]:
correct = 0
total = 0

with torch.no_grad(): 
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum().item() 

print(f'accuracy of 10000 test images: {100*correct/total}%')

accuracy of 10000 test images: 64.57%
